# 1_Import

In [1]:
import time
import json 
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns

In [2]:
with open(r"C:\Users\manon\Dev\2023_boissons\beverage_dispenser.json") as f:
    data = json.load(f)

In [3]:
# impression uniquement des 10 premieres lignes 
for x in data["content"][:10] : 
    print (x)

{'date': '2019-11-14T11:53:35.000Z', 'product': 'refill', 'machine': 'A'}
{'date': '2019-11-14T12:06:35.000Z', 'product': 'coffee', 'machine': 'A'}
{'date': '2019-11-14T12:07:59.000Z', 'product': 'soda', 'machine': 'A'}
{'date': '2019-11-14T12:13:45.000Z', 'product': 'soda', 'machine': 'A'}
{'date': '2019-11-14T12:22:35.000Z', 'product': 'coffee', 'machine': 'A'}
{'date': '2019-11-14T12:37:44.000Z', 'product': 'soda', 'machine': 'A'}
{'date': '2019-11-14T12:47:09.000Z', 'product': 'refill', 'machine': 'B'}
{'date': '2019-11-14T12:47:52.000Z', 'product': 'coffee', 'machine': 'A'}
{'date': '2019-11-14T12:51:00.000Z', 'product': 'coffee', 'machine': 'A'}
{'date': '2019-11-14T12:52:49.000Z', 'product': 'coffee', 'machine': 'A'}


# 2_ Conversion du JSON en DataFrame

In [4]:
df_init = pd.json_normalize(data["content"])

In [5]:
df_init

,date,product,machine
0,2019-11-14T11:53:35.000Z,refill,A
1,2019-11-14T12:06:35.000Z,coffee,A
2,2019-11-14T12:07:59.000Z,soda,A
3,2019-11-14T12:13:45.000Z,soda,A
4,2019-11-14T12:22:35.000Z,coffee,A
...,...,...,...
11037,2019-12-25T20:10:53.000Z,nrj,A
11038,2019-12-25T20:16:30.000Z,nrj,B
11039,2019-12-25T20:26:35.000Z,nrj,B
11040,2019-12-25T21:11:55.000Z,nrj,B


In [6]:
df_init = df_init.sort_values("date").copy()

In [7]:
items_machines = {"item" : ["coffee", "soda", "nrj"], 
                  "prix_de_vente" : [1, 1.6, 1.8],
                  "cout_achat" : [0.3, 0.6, 0.8], 
                  "capacite" : [280, 120, 60]}

In [8]:
fees = {"type_de_frais" : ["fixe", "a_ajouter_par_machine"],
         "nb_machines" : [4, 4],
         "deplacement_par_machine" : [50, 20],
         "emplacement_mensuel" : [0, 150],
         "entretien_mensuel" : [0, 120]}

In [9]:
items_machines["item"]

['coffee', 'soda', 'nrj']

## Suivi du stock dans le temps

In [10]:
df_init["machine"].unique()

array(['A', 'B', 'C', 'D'], dtype=object)

In [11]:
# Création d'un df par machine
machine_A = df_init[df_init["machine"]=="A"].reset_index(drop=True)
machine_B = df_init[df_init["machine"]=="B"].reset_index(drop=True)
machine_C = df_init[df_init["machine"]=="C"].reset_index(drop=True)
machine_D = df_init[df_init["machine"]=="D"].reset_index(drop=True)

In [12]:
# Création d'une liste avec le nom de chaque machine 
noms_des_machines = [machine_A, machine_B, machine_C, machine_D]

In [13]:
start = time.time()
for id_machine in noms_des_machines : # pour chaque machine
    for prod in items_machines["item"] : # pour chaque produit dans le dico des produits
        id_machine[str(prod)] = 0 # création d'une colonne au nom de chaque produit et affectation de la valeur 0 par defaut
        id_machine.loc[id_machine["product"] == "refill", prod] = items_machines["capacite"][items_machines["item"].index(prod)]   
        # ci dessus : affectatinon de la quantité maximale du produit par distributeur (définie dans le dico des produits)

    for i in range(len(id_machine)) : 
        if id_machine["product"].iat[i] == "refill" or i==0 :  # en cas de remplissage ou de la premiere ligne du df : on ne fait rien
            continue
        else : 
            produit_consomme = id_machine["product"].iat[i]  # recupération du produit acheté/consommé
            id_machine.iloc[i, 3:] = id_machine.iloc[i-1, 3:]    # copie des quantités en stock de la ligne précedente
            id_machine[produit_consomme].iat[i] -=1 # -1 sur le stock du produit acheté
print(time.time() - start)    

3.2644331455230713
